# MCS14
## Training the music separation model


In [ ]:
# @title 1. Install Dependencies
%%capture
# 1. Setup and install dependencies
!git clone https://github.com/Lim1802/MCS14_FIT3162_SINGING_VIDEO_SEPERATION.git
%cd /content/MCS14_FIT3162_SINGING_VIDEO_SEPERATION/demucs
!pip install -r requirements.txt

# Install soundtouch
!sudo apt-get update
!sudo apt-get install -y libsoundtouch-dev
!sudo apt-get install -y soundstretch


In [ ]:
# @title 2. Mount Google Drive (For training checkpoints and logs)
# 2. Mount Google Drive (for dataset storage and model checkpoints)
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# @title 3. Update paths to your dataset (Need to manually change)

#@markdown Dataset path (Assuming you uploaded the dataset to My Drive) & (Change "musdb18hq" to your dataset name)
dataset_path = '/content/drive/MyDrive/musdb18hq' #@param ["snippets:","/content/drive/MyDrive/musdb18hq","/content/drive/MyDrive/<your dataset name>"]{allow-input: true}
# 4. Update paths in configuration files
!sed -i "s|^  musdb:.*|  musdb: {dataset_path}|" conf/config.yaml
!sed -i 's|dir: outputs|dir: /content/drive/MyDrive/demucs_outputs|' conf/config.yaml
# Use a better architecture
!sed -i 's|model: demucs|model: hdemucs|' conf/config.yaml

In [ ]:
# @title 4. Set up Dora XP repository
%%capture
!tar xvf outputs.tar.gz

In [ ]:
# @title 5.1 Start training the model! (Hyper-Params will be further explained in our final report, for now just run this) (10 epochs will approximately take 10 hours)
!dora run -d \
    batch_size=16 \
    demucs.channels=32 \
    demucs.depth=4 \
    dset.segment=5 \
    misc.num_workers=2 \
    optim.lr=5e-4 \
    epochs=20

Executor: Starting 1 worker processes for DDP.
/usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  ret = run_job(
[10-12 17:17:09][demucs.train][INFO] - For logs, checkpoints and samples check /content/drive/MyDrive/demucs_outputs/xps/d7014c1b
[10-12 17:17:09][dora.distrib][INFO] - world_size is 1, skipping init.
100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [01:57<00:00,  1.18s/it]
[10-12 17:19:24][demucs.train][INFO] - train/valid set size: 18884 14
[10-12 17:19:24][demucs.solver][INFO] - ----------------------------------------------------------------------
[10-12 17:19:24][demucs.solver][INFO] - Training...
[10-12 17:43:52][demucs.solver][INFO] - Train | Epoch 1 | 295/1180 | 0.20 it/sec | Loss 0.1966 | Reco 0.1966 

In [ ]:
# @title 5.2 Continue training from checkpoint (Assuming you stopped halfway)

#@markdown Go to /content/drive/MyDrive/demucs_outputs/xps to find your model XP signature (e.g 97d170e1)
signature = "" # @param {"type":"string","placeholder":"Insert XP signature (e.g 97d170e1)"}
!dora run -d -f {signature}

In [ ]:
# @title 6. Export model! (After successfully exporting, your model's .th file will be located at /content/MCS14_FIT3162_SINGING_VIDEO_SEPERATION/demucs/release_models)
#@markdown Go to /content/drive/MyDrive/demucs_outputs/xps to find your model XP signature (e.g 97d170e1)
signature = "" # @param {"type":"string","placeholder":"Insert XP signature (e.g 97d170e1)"}

!python3 -m tools.export {signature}

INFO:__main__:Handling d7014c1b/batch_size=16 dem.channels=32 dem.depth=4 dse.segment=5 epochs=20 opt.lr=0.0005
INFO:dora.distrib:world_size is 1, skipping init.
INFO:demucs.train:train/valid set size: 18884 14
INFO:demucs.solver:Loading checkpoint model: /content/drive/MyDrive/demucs_outputs/xps/d7014c1b/checkpoint.th


# To test the model, create 3 folders in your My Drive: **Input, Output, Model**.
## Find your model's .th file located in /content/MCS14_FIT3162_SINGING_VIDEO_SEPERATION/demucs/release_models, and upload it in the **Model** folder.
## Upload a test song (.wav or .mp3) in the **Input** folder.
## Then run the code below, the separated track will be located in the **Output** folder.

In [ ]:
# @title 7. Test the model

#@markdown Go to /content/drive/MyDrive/Model to find your model XP signature (e.g 97d170e1)
signature = "d7014c1b" # @param {"type":"string","placeholder":"Insert XP signature (e.g 97d170e1)"}

import io
from pathlib import Path
import select
from shutil import rmtree
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO

from google.colab import files

# Define the input and output paths
in_path = '/content/drive/MyDrive/Input'
out_path = '/content/drive/MyDrive/Output'
# Define the Demucs model to use
model = signature # Put here the name of the model of the .th file in your case 'user-trained'

extensions = set("wav mp3".split())

def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out

def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

def separate(inp=None, outp=None):
    inp = inp or in_path
    outp = outp or out_path
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model, "--repo", "/content/drive/MyDrive/Model"]
    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")

separate()

In [ ]:
# @title 8. Evaluate the model
#@markdown Go to /content/MCS14_FIT3162_SINGING_VIDEO_SEPERATION/demucs/release_models to find your model XP signature (e.g 97d170e1)
signature = "d7014c1b" # @param {"type":"string","placeholder":"Insert XP signature (e.g 97d170e1)"}

!python3 -m tools.test_pretrained --repo ./release_models -n {signature}